In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install -q rouge-score
!pip install keras-nlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/cli/base_command.py", line 180, in _main
    status = self.run(options, args)
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/cli/req_command.py", line 199, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/commands/install.py", line 318, in run
    requirement_set = resolver.resolve(
  File "/usr/lo

In [ ]:
import keras_nlp
import numpy as np
import pandas as pd
import pathlib
import random
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.models import  model_from_json
from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert_vocab

In [ ]:
BATCH_SIZE = 128
EPOCHS = 5  # This should be at least 10 for convergence
MAX_SEQUENCE_LENGTH = 20
ENG_VOCAB_SIZE = 104890
SND_VOCAB_SIZE = 104890

EMBED_DIM = 256
INTERMEDIATE_DIM = 2048
NUM_HEADS = 8

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/English-to-Sindhi Translation/english-sindhi2.csv')

In [ ]:
text_pairs = data.values.tolist()

In [ ]:
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

104890 total pairs
73424 training pairs
15733 validation pairs
15733 test pairs


In [ ]:
def train_word_piece(text_samples, vocab_size, reserved_tokens):
    bert_vocab_args = dict(
        # The target vocabulary size
        vocab_size=vocab_size,
        # Reserved tokens that must be included in the vocabulary
        reserved_tokens=reserved_tokens,
        # Arguments for `text.BertTokenizer`
        bert_tokenizer_params={"lower_case": True},
    )

    word_piece_ds = tf.data.Dataset.from_tensor_slices(text_samples)
    vocab = bert_vocab.bert_vocab_from_dataset(
        word_piece_ds.batch(1000).prefetch(2), **bert_vocab_args
    )
    return vocab

In [ ]:
def train_word_piece(text_samples, vocab_size, reserved_tokens):
    bert_vocab_args = dict(
        # The target vocabulary size
        vocab_size=vocab_size,
        # Reserved tokens that must be included in the vocabulary
        reserved_tokens=reserved_tokens,
        # Arguments for `text.BertTokenizer`
        bert_tokenizer_params={"lower_case": True},
    )

    word_piece_ds = tf.data.Dataset.from_tensor_slices(text_samples)
    vocab = bert_vocab.bert_vocab_from_dataset(
        word_piece_ds.batch(1000).prefetch(2), **bert_vocab_args
    )
    return vocab

In [ ]:
reserved_tokens = ["[PAD]", "[UNK]", "[START]", "[END]"]

eng_samples = [text_pair[0] for text_pair in train_pairs]
eng_vocab = train_word_piece(eng_samples, ENG_VOCAB_SIZE, reserved_tokens)

snd_samples = [text_pair[1] for text_pair in train_pairs]
snd_vocab = train_word_piece(snd_samples, SND_VOCAB_SIZE, reserved_tokens)

In [ ]:
print("English Tokens: ", eng_vocab[100:110])
print("Sindhi Tokens: ", snd_vocab[100:110])

English Tokens:  ['out', 'if', 'just', 'one', 'didnt', 'going', 'an', 'from', 'would', 'good']
Sindhi Tokens:  ['ے', '۽', '۾', 'اهي', 'کي', 'مون', 'مان', 'توهان', 'جي', 'ته']


In [ ]:
eng_tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary=eng_vocab, lowercase=False
)
snd_tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary=snd_vocab, lowercase=False
)

In [ ]:
eng_input_ex = text_pairs[0][0]
eng_tokens_ex = eng_tokenizer.tokenize(eng_input_ex)
print("English sentence: ", eng_input_ex)
print("Tokens: ", eng_tokens_ex)
print("Recovered text after detokenizing: ", eng_tokenizer.detokenize(eng_tokens_ex))

print()

snd_input_ex = text_pairs[0][1]
snd_tokens_ex = snd_tokenizer.tokenize(snd_input_ex)
print("Sindhi sentence: ", snd_input_ex)
print("Tokens: ", snd_tokens_ex)
print("Recovered text after detokenizing: ", snd_tokenizer.detokenize(snd_tokens_ex))

English sentence:  i dont feel like taking a walk this morning
Tokens:  tf.Tensor([ 23  57 190  70 562  15 348  51 277], shape=(9,), dtype=int32)
Recovered text after detokenizing:  tf.Tensor(b'i dont feel like taking a walk this morning', shape=(), dtype=string)

Sindhi sentence:  مون کي اڄ صبح سير ڪرڻ لاء دل نه ٿو لڳي
Tokens:  tf.Tensor([ 105  104  190  342 1234  126  124  691  110  111  229], shape=(11,), dtype=int32)
Recovered text after detokenizing:  tf.Tensor(b'\xd9\x85\xd9\x88\xd9\x86 \xda\xa9\xd9\x8a \xd8\xa7\xda\x84 \xd8\xb5\xd8\xa8\xd8\xad \xd8\xb3\xd9\x8a\xd8\xb1 \xda\xaa\xd8\xb1\xda\xbb \xd9\x84\xd8\xa7\xd8\xa1 \xd8\xaf\xd9\x84 \xd9\x86\xd9\x87 \xd9\xbf\xd9\x88 \xd9\x84\xda\xb3\xd9\x8a', shape=(), dtype=string)


In [ ]:
def preprocess_batch(eng, snd):
    batch_size = tf.shape(snd)[0]

    eng = eng_tokenizer(eng)
    snd = snd_tokenizer(snd)

    # Pad `eng` to `MAX_SEQUENCE_LENGTH`.
    eng_start_end_packer = keras_nlp.layers.StartEndPacker(
        sequence_length=MAX_SEQUENCE_LENGTH,
        pad_value=eng_tokenizer.token_to_id("[PAD]"),
    )
    eng = eng_start_end_packer(eng)

    # Add special tokens (`"[START]"` and `"[END]"`) to `snd` and pad it as well.
    snd_start_end_packer = keras_nlp.layers.StartEndPacker(
        sequence_length=MAX_SEQUENCE_LENGTH + 1,
        start_value=snd_tokenizer.token_to_id("[START]"),
        end_value=snd_tokenizer.token_to_id("[END]"),
        pad_value=snd_tokenizer.token_to_id("[PAD]"),
    )
    snd = snd_start_end_packer(snd)

    return (
        {
            "encoder_inputs": eng,
            "decoder_inputs": snd[:, :-1],
        },
        snd[:, 1:],
    )


def make_dataset(pairs):
    eng_texts, snd_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    snd_texts = list(snd_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, snd_texts))
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.map(preprocess_batch, num_parallel_calls=tf.data.AUTOTUNE)
    return dataset.shuffle(2048).prefetch(16).cache()


train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [ ]:
# Encoder
encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")

x = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=ENG_VOCAB_SIZE,
    sequence_length=MAX_SEQUENCE_LENGTH,
    embedding_dim=EMBED_DIM,
    mask_zero=True,
)(encoder_inputs)

encoder_outputs = keras_nlp.layers.TransformerEncoder(
    intermediate_dim=INTERMEDIATE_DIM, num_heads=NUM_HEADS
)(inputs=x)
encoder = keras.Model(encoder_inputs, encoder_outputs)


# Decoder
decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, EMBED_DIM), name="decoder_state_inputs")

x = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=SND_VOCAB_SIZE,
    sequence_length=MAX_SEQUENCE_LENGTH,
    embedding_dim=EMBED_DIM,
    mask_zero=True,
)(decoder_inputs)

x = keras_nlp.layers.TransformerDecoder(
    intermediate_dim=INTERMEDIATE_DIM, num_heads=NUM_HEADS
)(decoder_sequence=x, encoder_sequence=encoded_seq_inputs)
x = keras.layers.Dropout(0.5)(x)
decoder_outputs = keras.layers.Dense(SND_VOCAB_SIZE, activation="softmax")(x)
decoder = keras.Model(
    [
        decoder_inputs,
        encoded_seq_inputs,
    ],
    decoder_outputs,
)
decoder_outputs = decoder([decoder_inputs, encoder_outputs])

transformer = keras.Model(
    [encoder_inputs, decoder_inputs],
    decoder_outputs,
    name="transformer",
)

In [ ]:
transformer.summary()
transformer.compile(
    "rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
hist = transformer.fit(train_ds, epochs=5, validation_data=val_ds)

Model: "transformer"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 token_and_position_embedding (  (None, None, 256)   26856960    ['encoder_inputs[0][0]']         
 TokenAndPositionEmbedding)                                                                       
                                                                                                  
 decoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 transformer_encoder (Transform  (None, None, 256)   1315072     ['token_and_position_em

In [ ]:
# serialize model to JSON
model_json = transformer.to_json()
with open("/content/drive/MyDrive/model_KerasNLP.json", "w") as json_file:
  json_file.write(model_json)
# serialize weights to HDF5
transformer.save_weights("/content/drive/MyDrive/model_KerasNLP.h5")
print("Saved model to disk")

Saved model to disk


In [ ]:
# load json and create model
json_file = open('/content/drive/MyDrive/model_KerasNLP.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json, custom_objects={'TokenAndPositionEmbedding': keras_nlp.layers.TokenAndPositionEmbedding,
                                                                  'TransformerEncoder': keras_nlp.layers.TransformerEncoder,
                                                                  'TransformerDecoder': keras_nlp.layers.TransformerDecoder})
# load weights into new model
model.load_weights("/content/drive/MyDrive/model_KerasNLP.h5")
print("Loaded model from disk")

Loaded model from disk


In [ ]:
def decode_sequences(input_sentences):
  batch_size = tf.shape(input_sentences)[0]

  # Tokenize the encoder input.
  encoder_input_tokens = eng_tokenizer(input_sentences).to_tensor(
    shape=(None, MAX_SEQUENCE_LENGTH)
  )

  # Define a function that outputs the next token's probability given the
  # input sequence.
  def token_probability_fn(decoder_input_tokens):
    return model([encoder_input_tokens, decoder_input_tokens])[:, -1, :]

  # Set the prompt to the "[START]" token.
  prompt = tf.fill((batch_size, 1), snd_tokenizer.token_to_id("[START]"))

  generated_tokens = keras_nlp.utils.greedy_search(
    token_probability_fn,
    prompt,
    max_length=20,
    end_token_id=snd_tokenizer.token_to_id("[END]"),
  )
  generated_sentences = snd_tokenizer.detokenize(generated_tokens)
  return generated_sentences


translated = decode_sequences(tf.constant(['this is dog']))
translated = translated.numpy()[0].decode("utf-8")
translated = (
    translated.replace("[PAD]", "")
    .replace("[START]", "")
    .replace("[END]", "")
    .strip()
)

In [ ]:
cd /content/drive/MyDrive/English-to-Sindhi Translation

/content/drive/MyDrive/English-to-Sindhi Translation


In [ ]:
print(translated)

هي هوس اهي


In [ ]:
!pip install flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import getpass

from pyngrok import ngrok, conf

print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth")
conf.get_default().auth_token = getpass.getpass()

# Open a TCP ngrok tunnel to the SSH server
connection_string = ngrok.connect(22, "tcp").public_url

ssh_url, port = connection_string.strip("tcp://").split(":")
print(f" * ngrok tunnel available, access with `ssh root@{ssh_url} -p{port}`")

Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth
··········
 * ngrok tunnel available, access with `ssh root@4.tcp.ngrok.io -p15448`


In [1]:
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, request
from wtforms import Form, TextAreaField, validators
import pickle
import os
import numpy as np
import keras_nlp
import numpy as np
import pandas as pd
import pathlib
import random
import tensorflow as tf
import matplotlib.pyplot as plt
import keras
from keras.models import  model_from_json
import getpass
from pyngrok import ngrok, conf
from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert_vocab

print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth")
conf.get_default().auth_token = getpass.getpass()

# Open a TCP ngrok tunnel to the SSH server
connection_string = ngrok.connect(22, "tcp").public_url

ssh_url, port = connection_string.strip("tcp://").split(":")
print(f" * ngrok tunnel available, access with `ssh root@{ssh_url} -p{port}`")

app = Flask(__name__)
run_with_ngrok(app)
app.config['SECRET_KEY'] = "abcmsdihgiosjawfjeioghaegnklaebn"



@app.route('/')
def index():
    return render_template('input_form.html')

def decode_sequences(input_sentences):
  batch_size = tf.shape(input_sentences)[0]

  # Tokenize the encoder input.
  encoder_input_tokens = eng_tokenizer(input_sentences).to_tensor(
    shape=(None, MAX_SEQUENCE_LENGTH)
  )

  # Define a function that outputs the next token's probability given the
  # input sequence.
  def token_probability_fn(decoder_input_tokens):
    return model([encoder_input_tokens, decoder_input_tokens])[:, -1, :]

  # Set the prompt to the "[START]" token.
  prompt = tf.fill((batch_size, 1), snd_tokenizer.token_to_id("[START]"))

  generated_tokens = keras_nlp.utils.greedy_search(
    token_probability_fn,
    prompt,
    max_length=20,
    end_token_id=snd_tokenizer.token_to_id("[END]"),
  )
  generated_sentences = snd_tokenizer.detokenize(generated_tokens)
  return generated_sentences



@app.route('/results', methods=['POST'])
def transate():
    text = request.form['text']

    translated = decode_sequences(tf.constant([text]))
    translated = translated.numpy()[0].decode("utf-8")
    translated = (
        translated.replace("[PAD]", "")
        .replace("[START]", "")
        .replace("[END]", "")
        .strip()
    )
    
    return render_template('results.html', predicted_text=translated)


app.run()


ModuleNotFoundError: ignored

In [ ]:
!pip install wtforms

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 136 kB 10.2 MB/s 


In [ ]:
!ngrok config add-authtoken 2Iar0FThbid00EdrNZaLJRCeYU5_4uktgMyy7EESP26TDnBTn

/bin/bash: ngrok: command not found


In [ ]:
!pip install flask==0.12.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 83 kB 1.2 MB/s 
  Attempting uninstall: flask
    Found existing installation: Flask 1.1.4
    Uninstalling Flask-1.1.4:
      Successfully uninstalled Flask-1.1.4


In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, request
from wtforms import Form, TextAreaField, validators
import numpy as np
import keras_nlp
import numpy as np
import pandas as pd
import pathlib
import random
import tensorflow as tf
import matplotlib.pyplot as plt
import keras
from keras.models import  model_from_json
from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert_vocab
import os
import threading
from flask import Flask
from pyngrok import ngrok

os.environ["FLASK_ENV"] = "development"

app = Flask(__name__)
port = 5000

# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(port).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url

@app.route('/')
def index():
    return render_template('input_form.html')

def decode_sequences(input_sentences):
  batch_size = tf.shape(input_sentences)[0]

  # Tokenize the encoder input.
  encoder_input_tokens = eng_tokenizer(input_sentences).to_tensor(
    shape=(None, MAX_SEQUENCE_LENGTH)
  )

  # Define a function that outputs the next token's probability given the
  # input sequence.
  def token_probability_fn(decoder_input_tokens):
    return model([encoder_input_tokens, decoder_input_tokens])[:, -1, :]

  # Set the prompt to the "[START]" token.
  prompt = tf.fill((batch_size, 1), snd_tokenizer.token_to_id("[START]"))

  generated_tokens = keras_nlp.utils.greedy_search(
    token_probability_fn,
    prompt,
    max_length=20,
    end_token_id=snd_tokenizer.token_to_id("[END]"),
  )
  generated_sentences = snd_tokenizer.detokenize(generated_tokens)
  return generated_sentences



@app.route('/results', methods=['POST'])
def transate():
    text = request.form['text']

    translated = decode_sequences(tf.constant([text]))
    translated = translated.numpy()[0].decode("utf-8")
    translated = (
        translated.replace("[PAD]", "")
        .replace("[START]", "")
        .replace("[END]", "")
        .strip()
    )
    
    return render_template('results.html', predicted_text=translated)




# Start the Flask server in a new thread
threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()

PyngrokNgrokHTTPError: ignored

In [ ]:
!pip install pyngrok


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install colabcode
!pip install fastapi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.3 MB 5.4 MB/s 
     |████████████████████████████████| 45 kB 3.6 MB/s 
     |████████████████████████████████| 9.8 MB 38.2 MB/s 
     |████████████████████████████████| 346 kB 48.4 MB/s 
     |████████████████████████████████| 423 kB 54.7 MB/s 
     |████████████████████████████████| 54 kB 2.6 MB/s 
     |████████████████████████████████| 58 kB 5.5 MB/s 
     |████████████████████████████████| 55 kB 3.8 MB/s 
     |████████████████████████████████| 273 kB 62.8 MB/s 
     |████████████████████████████████| 80 kB 8.4 MB/s 
     |████████████████████████████████| 133 kB 62.8 MB/s 
     |████████████████████████████████| 9.8 MB 39.0 MB/s 
     |████████████████████████████████| 9.8 MB 47.3 MB/s 
     |████████████████████████████████| 9.8 MB 41.9 MB/s 
     |████████████████████████████████| 9.8 MB 35.4 MB/s 
     |████████████████████████████████| 9.

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.2 MB/s 
     |████████████████████████████████| 64 kB 2.7 MB/s 


In [ ]:
from colabcode import ColabCode
from fastapi import FastAPI

In [ ]:
cc = ColabCode(port=12000, code=False)

In [ ]:
app = FastAPI()

@app.get('/')
async def read_root():
  return {'Hello' : 'World'}

In [ ]:
cc.run_app(app=app)

Public URL: NgrokTunnel: "https://456d-34-73-2-153.ngrok.io" -> "http://localhost:12000"


INFO:     Started server process [73]
INFO:uvicorn.error:Started server process [73]
INFO:     Waiting for application startup.
INFO:uvicorn.error:Waiting for application startup.
INFO:     Application startup complete.
INFO:uvicorn.error:Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:12000 (Press CTRL+C to quit)
INFO:uvicorn.error:Uvicorn running on http://127.0.0.1:12000 (Press CTRL+C to quit)


INFO:     121.52.154.73:0 - "GET / HTTP/1.1" 200 OK
INFO:     121.52.154.73:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found


INFO:     Shutting down
INFO:uvicorn.error:Shutting down
INFO:     Waiting for application shutdown.
INFO:uvicorn.error:Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:uvicorn.error:Application shutdown complete.
INFO:     Finished server process [73]
INFO:uvicorn.error:Finished server process [73]


In [ ]:
import os
import threading

from flask import Flask
from pyngrok import ngrok

os.environ["FLASK_ENV"] = "development"

app = Flask(__name__)
port = 5000

# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(port).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url

# ... Update inbound traffic via APIs to use the public-facing ngrok URL


# Define Flask routes
@app.route("/")
def index():
    return "Hello from Colab!"

# Start the Flask server in a new thread
threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()

PyngrokNgrokHTTPError: ignored